In [1]:
import pandas_datareader as web
import datetime
import numpy as np
import pandas as pd

In [2]:
# merge parts
import glob
tmp_list = glob.glob(r'stock_list*.csv')
tmp_list_dfs = []
for filename in tmp_list:
    if filename == "stock_list.csv" or filename == "stock_list2.csv" or filename == "stock_list3.csv" or filename == "stock_list4.csv" or filename == "stock_list5.csv":
        tmp = pd.read_csv(filename, engine='python')
        tmp = tmp.loc[:, ~tmp.columns.str.contains('^Unnamed')]
        tmp = tmp[tmp.adj_up_start_schedule_r_60.notnull()]
        tmp_list_dfs.append(tmp)
    
tmpall = pd.concat(tmp_list_dfs)
tmpall.drop_duplicates(subset ="symbol", keep="first", inplace=True)#subset ="symbol", keep="first", 
tmpall.to_csv(r'temp.csv')

In [8]:
# seperate good list from the bad
a = "stock_list"
b = "w"
t = pd.read_csv(r'{} - {}.csv'.format(a, b), engine='python')
t = t.loc[:, ~t.columns.str.contains('^Unnamed')]
# t = t.sort_values(['ups', 'avg'], t=[False, False])
# t.to_csv(r'{}.csv'.format(r'{} - {}.csv'.format(a, b)))
t[t.ignore == False].to_csv(r'saved/good/stock_good_list - {}.csv'.format(b), index=False)
t[t.ignore_error.notnull()].to_csv(r'saved/error/stock_err_list - {}.csv'.format(b), index=False)

In [3]:
# create main list
import glob
good_list = glob.glob(r'saved/good/*.csv')
good_list_dfs = []
for filename in good_list:
    tmp = pd.read_csv(filename, engine='python')
    tmp = tmp.loc[:, ~tmp.columns.str.contains('^Unnamed')]
    good_list_dfs.append(tmp)
    
main_list = pd.concat(good_list_dfs)
main_list.drop_duplicates(subset ="symbol", keep="first", inplace=True)
# main_list.sort_values(['close_up_start_schedule', 'close_up_percent_avg'], ascending=[True, False], inplace=True)
main_list.to_csv(r'main_stock_list.csv')

In [52]:
# update missing col in good list
tmp = pd.read_csv(r'saved/main/main_stock_list.csv', engine='python', index_col=['symbol'])
tmp = tmp.loc[:, ~tmp.columns.str.contains('^Unnamed')]
tmp

tmp2 = pd.read_csv(r'saved/good/stock_good_list - c.csv', engine='python', index_col=['symbol'])
tmp2 = tmp2.loc[:, ~tmp2.columns.str.contains('^Unnamed')]
tmp2
for i, row in tmp2.iterrows():
#     tmp2.at[i, tmp.index] = tmp.loc[i]
    try:
        tmp2.at[i, "close_ups"] = tmp.loc[i]["close_ups"]
        tmp2.at[i, "close_downs"] = tmp.loc[i]["close_downs"]
        tmp2.at[i, "close_up_start_schedule"] = tmp.loc[i]["close_up_start_schedule"]
        tmp2.at[i, "close_up_end_schedule"] = tmp.loc[i]["close_up_end_schedule"]
        tmp2.at[i, "close_up_percent_avg"] = tmp.loc[i]["close_up_percent_avg"]
    except:
        pass
    
tmp2.to_csv(r'saved/good/stock_good_list - c.csv')